<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# Understanding PyTorch Buffers

In essence, PyTorch buffers are tensor attributes associated with a PyTorch module or model similar to parameters, but unlike parameters, buffers are not updated during training.

Buffers in PyTorch are particularly useful when dealing with GPU computations, as they need to be transferred between devices (like from CPU to GPU) alongside the model's parameters. Unlike parameters, buffers do not require gradient computation, but they still need to be on the correct device to ensure that all computations are performed correctly.

In chapter 3, we use PyTorch buffers via `self.register_buffer`, which is only briefly explained in the book. Since the concept and purpose are not immediately clear, this code notebook offers a longer explanation with a hands-on example.

`register_parameter` 和 `register_buffer` 是 PyTorch 中 `nn.Module` 类的两个方法，它们用于将变量注册为模块的参数或缓存（buffer）。这两个方法的主要区别在于它们的目的和处理方式：

### 1. **`register_parameter`**

`register_parameter` 用于将一个张量注册为模型的 **可学习参数**，即在训练过程中会进行优化的参数。它通常用于权重、偏置等需要训练和更新的变量。

#### 用法：
```python
register_parameter(name, param)
```

- **`name`**：参数的名称（字符串），用于区分不同的参数。
- **`param`**：一个 `torch.Tensor` 对象，通常是 `nn.Parameter` 类型。`nn.Parameter` 是 `torch.Tensor` 的子类，表示该张量是模型的参数，会在优化过程中更新。

#### 示例：
```python
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 注册一个名为 'weight' 的参数
        self.register_parameter('weight', nn.Parameter(torch.randn(2, 2)))

    def forward(self, x):
        return torch.matmul(x, self.weight)

model = MyModel()
print(model.weight)
```

在这个例子中，`weight` 被注册为一个需要优化的参数，在训练过程中会更新。

#### 特点：
- `register_parameter` 注册的变量会出现在 `model.parameters()` 中，表示它是模型的参数。
- 这些参数会在训练过程中通过优化算法（如 SGD、Adam）进行更新。

### 2. **`register_buffer`**

`register_buffer` 用于将一个张量注册为 **缓存**。缓存是模型中需要保存但 **不需要优化** 的变量。这些变量在模型中进行前向传播时可能需要使用，但它们不需要通过反向传播来更新。

#### 用法：
```python
register_buffer(name, tensor)
```

- **`name`**：缓存的名称（字符串），用于区分不同的缓存。
- **`tensor`**：一个 `torch.Tensor` 对象，表示缓存数据，通常是常量或不需要优化的状态变量。

#### 示例：
```python
import torch
import torch.nn as nn

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        # 注册一个名为 'running_mean' 的缓存
        self.register_buffer('running_mean', torch.zeros(2))

    def forward(self, x):
        # 使用缓存变量
        return x + self.running_mean

model = MyModel()
print(model.running_mean)
```

在这个例子中，`running_mean` 是一个缓存变量，它在前向传播时使用，但不会被更新（不参与反向传播）。

#### 特点：
- `register_buffer` 注册的变量不会出现在 `model.parameters()` 中，因此它不会被优化器更新。
- 缓存通常用于保存一些模型状态，比如 BN（Batch Normalization）中的均值和方差，或者其他一些常量值。

### 3. **区别**：

| 特性               | `register_parameter`                           | `register_buffer`                          |
|--------------------|-----------------------------------------------|-------------------------------------------|
| **用途**           | 注册一个需要训练的参数（如权重、偏置等）。      | 注册一个不需要训练的缓存（如常量或中间变量）。 |
| **是否更新**       | 是，模型会自动更新这个参数。                  | 否，缓存的内容不会通过反向传播更新。         |
| **是否出现在 `parameters()`** | 是，出现在模型的 `parameters()` 中。     | 否，不会出现在 `parameters()` 中。          |
| **示例**           | 权重、偏置等需要通过优化更新的张量。            | 批标准化中的 `running_mean` 或 `running_var`。 |

### 4. **实际应用场景**：

- `register_parameter`：
  - 用于注册模型的训练参数，如线性层的权重和偏置。
  - 例如，在自定义模型中，如果你要定义一个新的权重矩阵并希望它参与训练，则使用 `register_parameter`。

- `register_buffer`：
  - 用于注册模型中不需要更新的常量或缓存变量。一个典型的例子是 **Batch Normalization** 层中的 `running_mean` 和 `running_var`，它们用于记录训练过程中计算的均值和方差，这些值不会更新，但需要在模型的保存和加载中保存。
  - 例如，在自定义模型中，如果你需要保存模型的一些状态（例如累积的统计信息、常量等），则使用 `register_buffer`。

### 示例：Batch Normalization 中的使用

```python
import torch
import torch.nn as nn

class MyBatchNorm(nn.Module):
    def __init__(self, num_features):
        super(MyBatchNorm, self).__init__()
        self.num_features = num_features
        # 训练时的均值和方差
        self.register_parameter('weight', nn.Parameter(torch.ones(num_features)))  # scale
        self.register_parameter('bias', nn.Parameter(torch.zeros(num_features)))    # shift
        
        # 不参与训练的均值和方差（缓存）
        self.register_buffer('running_mean', torch.zeros(num_features))  # 缓存均值
        self.register_buffer('running_var', torch.ones(num_features))   # 缓存方差

    def forward(self, x):
        # 在训练时使用 running_mean 和 running_var
        return (x - self.running_mean) / (self.running_var + 1e-5)**0.5 * self.weight + self.bias

# 创建模型
model = MyBatchNorm(10)
print(model.running_mean)  # 输出缓存的 running_mean
print(model.weight)        # 输出训练参数 weight
```

在这个例子中，`weight` 和 `bias` 是通过 `register_parameter` 注册的训练参数，而 `running_mean` 和 `running_var` 是通过 `register_buffer` 注册的缓存变量，不参与训练，但会在模型的保存和加载时被保存。

### 总结：
- **`register_parameter`** 用于注册需要训练的参数（如权重、偏置），这些参数会在模型训练过程中被更新。
- **`register_buffer`** 用于注册不需要训练的缓存变量（如统计量、常量），这些变量不会被优化器更新，但会随着模型一起保存和加载。

## An example without buffers

Suppose we have the following code, which is based on code from chapter 3. This version has been modified to exclude buffers. It implements the causal self-attention mechanism used in LLMs:

In [1]:
import torch
import torch.nn as nn

class CausalAttentionWithoutBuffers(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

We can initialize and run the module as follows on some example data:

In [2]:
torch.manual_seed(123)

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

batch = torch.stack((inputs, inputs), dim=0)
context_length = batch.shape[1]
d_in = inputs.shape[1]
d_out = 2

ca_without_buffer = CausalAttentionWithoutBuffers(d_in, d_out, context_length, 0.0)

with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]])


So far, everything has worked fine so far.

However, when training LLMs, we typically use GPUs to accelerate the process. Therefore, let's transfer the `CausalAttentionWithoutBuffers` module onto a GPU device.

Please note that this operation requires the code to be run in an environment equipped with GPUs.

In [3]:
print("Machine has GPU:", torch.cuda.is_available())

batch = batch.to("cuda")
ca_without_buffer.to("cuda");

Machine has GPU: True


Now, let's run the code again:

In [4]:
with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

RuntimeError: expected self and mask to be on the same device, but got mask on cpu and self on cuda:0

Running the code resulted in an error. What happened? It seems like we attempted a matrix multiplication between a tensor on a GPU and a tensor on a CPU. But we moved the module to the GPU!?


Let's double-check the device locations of some of the tensors:

In [5]:
print("W_query.device:", ca_without_buffer.W_query.weight.device)
print("mask.device:", ca_without_buffer.mask.device)

W_query.device: cuda:0
mask.device: cpu


In [6]:
type(ca_without_buffer.mask)

torch.Tensor

As we can see, the `mask` was not moved onto the GPU. That's because it's not a PyTorch parameter like the weights (e.g., `W_query.weight`).

This means we  have to manually move it to the GPU via `.to("cuda")`:

In [7]:
ca_without_buffer.mask = ca_without_buffer.mask.to("cuda")
print("mask.device:", ca_without_buffer.mask.device)

mask.device: cuda:0


Let's try our code again:

In [8]:
with torch.no_grad():
    context_vecs = ca_without_buffer(batch)

print(context_vecs)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], device='cuda:0')


This time, it worked!

However, remembering to move individual tensors to the GPU can be tedious. As we will see in the next section, it's easier to use `register_buffer` to register the `mask` as a buffer.

## An example with buffers

Let's now modify the causal attention class to register the causal `mask` as a buffer:

In [9]:
import torch
import torch.nn as nn

class CausalAttentionWithBuffer(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        # Old:
        # self.mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)

        # New:
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)

        context_vec = attn_weights @ values
        return context_vec

Now, conveniently, if we move the module to the GPU, the mask will be located on the GPU as well:

In [10]:
ca_with_buffer = CausalAttentionWithBuffer(d_in, d_out, context_length, 0.0)
ca_with_buffer.to("cuda")

print("W_query.device:", ca_with_buffer.W_query.weight.device)
print("mask.device:", ca_with_buffer.mask.device)

W_query.device: cuda:0
mask.device: cuda:0


In [11]:
with torch.no_grad():
    context_vecs = ca_with_buffer(batch)

print(context_vecs)

tensor([[[0.4772, 0.1063],
         [0.5891, 0.3257],
         [0.6202, 0.3860],
         [0.5478, 0.3589],
         [0.5321, 0.3428],
         [0.5077, 0.3493]],

        [[0.4772, 0.1063],
         [0.5891, 0.3257],
         [0.6202, 0.3860],
         [0.5478, 0.3589],
         [0.5321, 0.3428],
         [0.5077, 0.3493]]], device='cuda:0')


As we can see above, registering a tensor as a buffer can make our lives a lot easier: We don't have to remember to move tensors to a target device like a GPU manually.

## Buffers and `state_dict`

- Another advantage of PyTorch buffers, over regular tensors, is that they get included in a model's `state_dict`
- For example, consider the `state_dict` of the causal attention object without buffers

In [12]:
ca_without_buffer.state_dict()

OrderedDict([('W_query.weight',
              tensor([[-0.2354,  0.0191, -0.2867],
                      [ 0.2177, -0.4919,  0.4232]], device='cuda:0')),
             ('W_key.weight',
              tensor([[-0.4196, -0.4590, -0.3648],
                      [ 0.2615, -0.2133,  0.2161]], device='cuda:0')),
             ('W_value.weight',
              tensor([[-0.4900, -0.3503, -0.2120],
                      [-0.1135, -0.4404,  0.3780]], device='cuda:0'))])

- The mask is not included in the `state_dict` above
- However, the mask *is* included in the `state_dict` below, thanks to registering it as a buffer

In [13]:
ca_with_buffer.state_dict()

OrderedDict([('mask',
              tensor([[0., 1., 1., 1., 1., 1.],
                      [0., 0., 1., 1., 1., 1.],
                      [0., 0., 0., 1., 1., 1.],
                      [0., 0., 0., 0., 1., 1.],
                      [0., 0., 0., 0., 0., 1.],
                      [0., 0., 0., 0., 0., 0.]], device='cuda:0')),
             ('W_query.weight',
              tensor([[-0.1362,  0.1853,  0.4083],
                      [ 0.1076,  0.1579,  0.5573]], device='cuda:0')),
             ('W_key.weight',
              tensor([[-0.2604,  0.1829, -0.2569],
                      [ 0.4126,  0.4611, -0.5323]], device='cuda:0')),
             ('W_value.weight',
              tensor([[ 0.4929,  0.2757,  0.2516],
                      [ 0.2377,  0.4800, -0.0762]], device='cuda:0'))])

- A `state_dict` is useful when saving and loading trained PyTorch models, for example
- In this particular case, saving and loading the `mask` is maybe not super useful, because it remains unchanged during training; so, for demonstration purposes, let's assume it was modified where all `1`'s were changed to `2`'s:

In [14]:
ca_with_buffer.mask[ca_with_buffer.mask == 1.] = 2.
ca_with_buffer.mask

tensor([[0., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2.],
        [0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0.]], device='cuda:0')

- Then, if we save and load the model, we can see that the mask is restored with the modified value

In [15]:
torch.save(ca_with_buffer.state_dict(), "model.pth")

new_ca_with_buffer = CausalAttentionWithBuffer(d_in, d_out, context_length, 0.0)
new_ca_with_buffer.load_state_dict(torch.load("model.pth"))

new_ca_with_buffer.mask

tensor([[0., 2., 2., 2., 2., 2.],
        [0., 0., 2., 2., 2., 2.],
        [0., 0., 0., 2., 2., 2.],
        [0., 0., 0., 0., 2., 2.],
        [0., 0., 0., 0., 0., 2.],
        [0., 0., 0., 0., 0., 0.]])

- This is not true if we don't use buffers:

In [16]:
ca_without_buffer.mask[ca_without_buffer.mask == 1.] = 2.

torch.save(ca_without_buffer.state_dict(), "model.pth")

new_ca_without_buffer = CausalAttentionWithoutBuffers(d_in, d_out, context_length, 0.0)
new_ca_without_buffer.load_state_dict(torch.load("model.pth"))

new_ca_without_buffer.mask

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])